In [2]:
import os
from glob import glob
from tqdm import tqdm

# speaker encoder
from CookieTTS.utils.dataset.autovc_speaker_encoder.make_metadata import get_speaker_encoder    
speaker_encoder = get_speaker_encoder().cuda()

def get_speaker_embed(path):
    if path.endswith('.pt'):# if pytorch file
        return torch.load(path).float()
    elif path.endswith('.npy'):# if numpy file
        return torch.from_numpy(np.load(path)).float()
    else:
        return speaker_encoder.get_embed_from_path(path).float()

DATASET_FOLDER = r'I:\TTS\HiFiDatasets'

speaker_embeddings = {}
for speaker_folder in os.listdir(DATASET_FOLDER):
    if speaker_folder not in speaker_embeddings:
        speaker_embeddings[speaker_folder] = []
    audio_files = glob(os.path.join(DATASET_FOLDER, speaker_folder, '*.wav'))[:32]
    for audio_file in  tqdm(audio_files):
        speaker_embeddings[speaker_folder].append(get_speaker_embed(audio_file).cpu())# .append([embed])

print(len(list(speaker_embeddings.items())[0]))

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  5.93it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  6.14it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:04<00:00,  7.78it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
 84%|█████████████████████████████████████████████████████████████████████▏            | 

KeyboardInterrupt: 

In [7]:
for speaker_folder in speaker_embeddings.keys():
    speaker_embeddings[speaker_folder] = torch.stack(speaker_embeddings[speaker_folder], dim=0)# [B, embed]

In [6]:
import torch

cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

In [10]:
speaker_embeddings_shuffled = {k: v[torch.randperm(v.shape[0])] for k, v in speaker_embeddings.items()}

In [22]:
print("Similarity with same Speaker")
for speaker in speaker_embeddings.keys():
    print(f'{speaker} vs {speaker}')
    diff = cos(speaker_embeddings[speaker], speaker_embeddings_shuffled[speaker]).mean()
    print(diff)
    print("\n")

Similarity with same Speaker
VCTK__p225 vs VCTK__p225
tensor(0.6569, grad_fn=<MeanBackward0>)


VCTK__p226 vs VCTK__p226
tensor(0.6543, grad_fn=<MeanBackward0>)


VCTK__p227 vs VCTK__p227
tensor(0.7193, grad_fn=<MeanBackward0>)


VCTK__p228 vs VCTK__p228
tensor(0.8082, grad_fn=<MeanBackward0>)




In [24]:
# p225 = Female
# p226 = Male
# p227 = Male
# p228 = Female

print("Similarity with different Speaker")
speaker_list = list(speaker_embeddings.keys())[-1:]+list(speaker_embeddings.keys())[:-1]
speaker_shifted = {speaker: speaker_list[i] for i, speaker in enumerate(speaker_embeddings.keys())}
for speaker in speaker_embeddings.keys():
    print(f'{speaker} vs {speaker_shifted[speaker]}')
    diff = cos(speaker_embeddings[speaker], speaker_embeddings_shuffled[speaker_shifted[speaker]]).mean()
    print(diff)
    print("\n")

Similarity with different Speaker
VCTK__p225 vs VCTK__p228
tensor(0.4316, grad_fn=<MeanBackward0>)


VCTK__p226 vs VCTK__p225
tensor(0.2719, grad_fn=<MeanBackward0>)


VCTK__p227 vs VCTK__p226
tensor(0.5217, grad_fn=<MeanBackward0>)


VCTK__p228 vs VCTK__p227
tensor(0.2222, grad_fn=<MeanBackward0>)


